# Análise Eleitoral

### 0) Configuração dos bancos a serem utilizados

In [1]:
import pandas as pd
import tse

In [2]:
CANDIDATO = 'BOCALOM'
ESTADO = 'ac' # lowercase!

In [3]:
database_1 = f'databases/{ESTADO}_2024.sql'
database_2 = 'databases/eleicoes_2024.sql'

In [4]:
analise = tse.TseAnalysis(
    database={
        'path': database_1,
        'alias': f'{ESTADO}'
    })

In [5]:
analise.attach_database(
    attach= [
        {
            'path': database_2,
            'alias': 'geral'
        }
    ])

Bancos carregados:
0) main
-path: d:\projects\tse-data\databases\ac_2024.sql
--tables: 
-- resultados
-- eleitorado
-----
2) ac
-path: d:\projects\tse-data\databases\ac_2024.sql
--tables: 
-- resultados
-- eleitorado
-----
3) geral
-path: d:\projects\tse-data\databases\eleicoes_2024.sql
--tables: 
-- locais
-----


In [6]:
analise.create_index(
    table='resultados',
    columns=['NM_VOTAVEL', 'NR_LOCAL_VOTACAO', 'NR_ZONA', 'NR_SECAO', 'CD_MUNICIPIO']
)
analise.create_index(
    table='eleitorado',
    columns=['NR_LOCAL_VOTACAO', 'NR_ZONA', 'NR_SECAO', 'CD_MUNICIPIO']
)

'Indices criados com sucesso'

---

### 1) Criação de tabela com perfil de eleitorado por seção

In [7]:
# a função generate_stats_query foi escrita para gerar uma query específica para a tabela {ESTADO}. eleitorado
# ela analisa os valores distintos de cada coluna argumento e produz uma query que soma a quantidade de aparições para cada caso
q = analise.generate_stats_query(
    alias=f'{ESTADO}',
    table='eleitorado',
    cols=[
        'DS_GRAU_ESCOLARIDADE',
        'DS_RACA_COR',
        'DS_FAIXA_ETARIA',
        'DS_GENERO'
    ]
)

In [8]:
eleitorado_secao = pd.read_sql(q, analise.conn)

In [9]:
eleitorado_secao.head(2) # este DataFrame expressa a quantidade de eleitores por seçao divido em categorias

,municipio,local,zona,secao,ds_grau_escolaridade_ensino_fundamental_completo,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_analfabeto,ds_grau_escolaridade_ensino_médio_completo,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_lê_e_escreve,...,ds_faixa_etaria_70_a_74_anos,ds_faixa_etaria_65_a_69_anos,ds_faixa_etaria_35_a_39_anos,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_60_a_64_anos,ds_faixa_etaria_30_a_34_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino
0,1023,1015,1,576,12,53,19,51,15,54,...,3,10,26,0,0,20,34,0,118,155
1,1023,1015,1,577,16,52,33,68,16,36,...,3,16,26,0,0,13,29,0,120,155


In [10]:
q_2 = f"""
    SELECT
        CD_MUNICIPIO as municipio,
        NR_LOCAL_VOTACAO as local,
        NR_ZONA as zona,
        NR_SECAO as secao,
        QT_ELEITOR_SECAO as eleitores_totais
    FROM
        geral.locais
    WHERE
        SG_UF = '{ESTADO.upper()}'
"""

In [11]:
eleitores_secao = pd.read_sql(q_2, analise.conn)

In [12]:
eleitores_secao.head(2) # complementa o DataFrame anterior com a quantidade total de eleitores na seção

,municipio,local,zona,secao,eleitores_totais
0,1392,3514,1,840,98
1,1392,2054,9,468,93


In [13]:
eleitorado_secao = pd.merge(
    eleitorado_secao,
    eleitores_secao,
    how='inner',
    on=[
        'municipio',
        'local',
        'zona',
        'secao'
    ]
)

In [14]:
eleitorado_secao.head(2) # resultado obtido

,municipio,local,zona,secao,ds_grau_escolaridade_ensino_fundamental_completo,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_analfabeto,ds_grau_escolaridade_ensino_médio_completo,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_lê_e_escreve,...,ds_faixa_etaria_65_a_69_anos,ds_faixa_etaria_35_a_39_anos,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_60_a_64_anos,ds_faixa_etaria_30_a_34_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino,eleitores_totais
0,1023,1015,1,576,12,53,19,51,15,54,...,10,26,0,0,20,34,0,118,155,273
1,1023,1015,1,577,16,52,33,68,16,36,...,16,26,0,0,13,29,0,120,155,275


---

### 2) Cria tabela com apuração dos resultados por seção de determinado candidato

In [15]:
query = f"""
    SELECT
        NM_VOTAVEL as candidato,
        CD_MUNICIPIO as municipio,
        NR_LOCAL_VOTACAO as local,
        NR_ZONA as zona,
        NR_SECAO as secao,
        NR_TURNO as turno,
        QT_VOTOS as votos_recebidos
    FROM
        {ESTADO}.resultados
    WHERE
        NM_VOTAVEL LIKE '{'%' + CANDIDATO.replace(' ', '%') + '%'}'
"""

In [16]:
resultado_secao = pd.read_sql(query, analise.conn)

In [17]:
resultado_secao.head(2)

,candidato,municipio,local,zona,secao,turno,votos_recebidos
0,SEBASTIAO BOCALOM RODRIGUES,1392,1953,9,314,1,146
1,SEBASTIAO BOCALOM RODRIGUES,1392,1465,1,828,1,114


In [18]:
resultado_perfil = pd.merge(
    resultado_secao,
    eleitorado_secao,
    how='inner',
    on=['municipio', 'local', 'zona', 'secao']
)

In [19]:
resultado_perfil.head(2)

,candidato,municipio,local,zona,secao,turno,votos_recebidos,ds_grau_escolaridade_ensino_fundamental_completo,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_analfabeto,...,ds_faixa_etaria_65_a_69_anos,ds_faixa_etaria_35_a_39_anos,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_60_a_64_anos,ds_faixa_etaria_30_a_34_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino,eleitores_totais
0,SEBASTIAO BOCALOM RODRIGUES,1392,1953,9,314,1,146,18,83,14,...,5,24,0,0,16,37,0,181,156,337
1,SEBASTIAO BOCALOM RODRIGUES,1392,1465,1,828,1,114,1,28,2,...,2,7,0,0,3,8,0,68,58,126


In [20]:
resultado_perfil['percentual_votos'] = resultado_perfil['votos_recebidos']/resultado_perfil['eleitores_totais']

In [21]:
nome = resultado_perfil['candidato'][0]
resultado_perfil.to_csv(f'resultados_secao_absolutos/{nome.replace(' ', '_')}.csv')